In [1]:
import urllib.request as urq
import sys
from bs4 import BeautifulSoup
import csv
import copy
import os
import threading
import time
import datetime

In [24]:
user_dir_path = "/Users/dohk/GoogleDrive/coinCrawling/Bithumb_Data_Collector"
save_file_name = "coin_{0}.csv"
bithumb_graph_url = 'http://index.bithumb.com/api/bithumb/localAPI.php?api=graph&coin={0}&back={1}'
coin_list = ['BTC', 'ETH', 'XRP', 'BCH', 'LTC', 'EOS', 'DASH', 'XMR', 'ETC', 'QTUM', 'BTG', 'ZEC']

all_coin_msec_ago = dict() #얼마나 이전의 데이터를 가져올지
all_data_list = dict() #전체 데이터
for coin in coin_list:
    all_data_list[coin] = list()
    all_coin_msec_ago[coin] = 0
current_data_list = list() #현재 수집 데이터
long_msec_for_back_to = 3600000000 #1000시간
much_long_msec_for_back_to = 90000000 #25시간

loop_sleep_time = 3

In [12]:
def dataParser(coin, msec):
    if coin in coin_list:
        pass
    else:
        print('The coin({0}) you wrote is not on coin-list!!'.format(coin))
        return False
    try:
        html = urq.urlopen(bithumb_graph_url.format(coin, msec))
    except:
        print("URL_OPEN_ERROR")
        return False
    data = BeautifulSoup(html.read(), "html.parser")
    data_list = list()
    text_data = data.text
    text_data = text_data[text_data.find('[')+1:text_data.rfind(']')]
    l_index = 0
    r_index = 0
    append_l_flag = False
    append_r_flag = False
    for index, letter in enumerate(text_data):
        if letter == '[':
            l_index = index
            append_l_flag = True
        elif letter == ']':
            r_index = index
            append_r_flag = True
        if append_l_flag and append_r_flag:
            row = text_data[l_index+1:r_index].split(',')
            row.insert(1, datetime.datetime.fromtimestamp(int(int(row[0])/1000)).strftime('%Y-%m-%d %H:%M:%S'))
            data_list.append(row)
            append_l_flag = False
            append_r_flag = False
    return data_list


In [13]:
# Loop 1
def longBackTo(coin):
    print(coin + " - Loop 1 start at " + str(datetime.datetime.now()))
    time.sleep(loop_sleep_time)
    global current_data_list
    global all_data_list
    global all_coin_msec_ago
    current_data_list = dataParser(coin, str(all_coin_msec_ago[coin]))
    if not current_data_list:
        print("Can't search long ago data - {0}.".format(coin))
        print(coin + " - Loop 1 end")
        return False
    if len(all_data_list[coin]) == 0:
        all_data_list[coin] = current_data_list
        #더 이전의 데이터를 가져오기 위함
        all_coin_msec_ago[coin] = all_coin_msec_ago[coin] + long_msec_for_back_to
        print(coin + " - Loop 1 end")
        return current_data_list
    else:
        #row[0] = 현재 수집된 데이터 한 줄의 시간
        for row in current_data_list:
            if int(all_data_list[coin][0][0]) > int(row[0]):
                all_data_list[coin].insert(0, row)
            elif int(all_data_list[coin][-1][0]) < int(row[0]):
                all_data_list[coin].append(row)
            else:
                for i, a_row in enumerate(all_data_list[coin][0:-1]):
                    if int(a_row[0]) == int(row[0]):
                        break
                    elif int(a_row[0]) < int(row[0]) and int(all_data_list[coin][i+1][0]) > int(row[0]):
                        all_data_list[coin].insert(i+1, row)
                        break
        #더 이전의 데이터를 가져오기 위함
        all_coin_msec_ago[coin] = all_coin_msec_ago[coin] + long_msec_for_back_to
    print(coin + " - Loop 1 end")


In [14]:
# Loop 2
def muchLongBackTo(coin):
    print(coin + " - Loop 2 start at " + str(datetime.datetime.now()))
    time.sleep(loop_sleep_time)
    global current_data_list
    global all_data_list
    global all_coin_msec_ago
    current_data_list = dataParser(coin, str(all_coin_msec_ago[coin] + much_long_msec_for_back_to))
    if not current_data_list:
        print("Can't search much long ago data - {0}.".format(coin))
        print(coin + " - Loop 2 end")
        return False
    if len(all_data_list[coin]) == 0:
        all_data_list[coin] = current_data_list
        #더욱 더 이전의 데이터를 가져오기 위함
        all_coin_msec_ago[coin] = all_coin_msec_ago[coin] + much_long_msec_for_back_to
        print(coin + " - Loop 2 end")
        return current_data_list
    else:
        #row[0] = 현재 수집된 데이터 한 줄의 시간
        for row in current_data_list:
            if int(all_data_list[coin][0][0]) > int(row[0]):
                all_data_list[coin].insert(0, row)
            elif int(all_data_list[coin][-1][0]) < int(row[0]):
                all_data_list[coin].append(row)
            else:
                for i, a_row in enumerate(all_data_list[coin][0:-1]):
                    if int(a_row[0]) == int(row[0]):
                        break
                    elif int(a_row[0]) < int(row[0]) and int(all_data_list[coin][i+1][0]) > int(row[0]):
                        all_data_list[coin].insert(i+1, row)
                        break
        #더욱 더 이전의 데이터를 가져오기 위함
        all_coin_msec_ago[coin] = all_coin_msec_ago[coin] + much_long_msec_for_back_to
    print(coin + " - Loop 2 end")


In [15]:
# Loop 3
def currentOnly(coin):
    print(coin + " - Loop 3 start at " + str(datetime.datetime.now()))
    time.sleep(loop_sleep_time)
    global current_data_list
    global all_data_list
    current_data_list = dataParser(coin, str(0))
    if not current_data_list:
        print("Can't search long ago data - {0}.".format(coin))
        print(coin + " - Loop 3 end")
        return False
    if len(all_data_list[coin]) == 0:
        all_data_list[coin] = current_data_list
        print(coin + " - Loop 3 end")
        return current_data_list
    else:
        #row[0] = 현재 수집된 데이터 한 줄의 시간
        for row in current_data_list:
            if int(all_data_list[coin][0][0]) > int(row[0]):
                all_data_list[coin].insert(0, row)
            elif int(all_data_list[coin][-1][0]) < int(row[0]):
                all_data_list[coin].append(row)
            else:
                for i, a_row in enumerate(all_data_list[coin][0:-1]):
                    if int(a_row[0]) == int(row[0]):
                        break
                    elif int(a_row[0]) < int(row[0]) and int(all_data_list[coin][i+1][0]) > int(row[0]):
                        all_data_list[coin].insert(i+1, row)
                        break
    print(coin + " - Loop 3 end")


In [16]:
# Loop 4
def saveAllDataList(coin):
    print(coin + " - Saving data start at " + str(datetime.datetime.now()))
    time.sleep(loop_sleep_time)
    f = open(save_file_name.format(coin), 'w', encoding='utf-8', newline='')
    wr = csv.writer(f)
    for line in all_data_list[coin]:
        wr.writerow(line)
    f.close()
    print(coin + " - Saved data")

In [17]:
def openAllDataList(coin):
    global all_data_list
    f = open(save_file_name.format(coin), 'r', encoding='utf-8')
    rdr = csv.reader(f)
    for line in rdr:
        all_data_list[coin].append(line)
    f.close()

In [25]:
if __name__ == "__main__":
    timer = 0
    for coin in coin_list:
        if os.path.exists(user_dir_path + save_file_name.format(coin)):
            openAllDataList(coin)
    while True:
        for coin in coin_list:
            longBackTo(coin)
            if timer % 2 == 1:
                muchLongBackTo(coin)
                currentOnly(coin)
            saveAllDataList(coin)
        
        timer = timer + 1

BTC - Loop 1 start at 2018-03-19 12:23:41.144513
BTC - Loop 1 end
BTC - Saving data start at 2018-03-19 12:23:44.408431
BTC - Saved data
ETH - Loop 1 start at 2018-03-19 12:23:47.427166
ETH - Loop 1 end
ETH - Saving data start at 2018-03-19 12:23:51.325977
ETH - Saved data
XRP - Loop 1 start at 2018-03-19 12:23:54.348975
XRP - Loop 1 end
XRP - Saving data start at 2018-03-19 12:23:57.547532
XRP - Saved data
BCH - Loop 1 start at 2018-03-19 12:24:00.565733
BCH - Loop 1 end
BCH - Saving data start at 2018-03-19 12:24:03.913231
BCH - Saved data
LTC - Loop 1 start at 2018-03-19 12:24:06.931587
LTC - Loop 1 end
LTC - Saving data start at 2018-03-19 12:24:10.171588
LTC - Saved data
EOS - Loop 1 start at 2018-03-19 12:24:13.190048
EOS - Loop 1 end
EOS - Saving data start at 2018-03-19 12:24:16.474942
EOS - Saved data
DASH - Loop 1 start at 2018-03-19 12:24:19.493791
DASH - Loop 1 end
DASH - Saving data start at 2018-03-19 12:24:22.730803
DASH - Saved data
XMR - Loop 1 start at 2018-03-19 12:2

EOS - Saved data
DASH - Loop 1 start at 2018-03-19 14:08:55.459631
DASH - Loop 1 end
DASH - Loop 2 start at 2018-03-19 14:09:45.421590
DASH - Loop 2 end
DASH - Loop 3 start at 2018-03-19 14:10:39.616557
DASH - Loop 3 end
DASH - Saving data start at 2018-03-19 14:16:14.680293
DASH - Saved data
XMR - Loop 1 start at 2018-03-19 14:16:17.753286
XMR - Loop 1 end
XMR - Loop 2 start at 2018-03-19 14:17:07.931711
XMR - Loop 2 end
XMR - Loop 3 start at 2018-03-19 14:17:38.905205
XMR - Loop 3 end
XMR - Saving data start at 2018-03-19 14:22:44.582562
XMR - Saved data
ETC - Loop 1 start at 2018-03-19 14:22:47.652340
ETC - Loop 1 end
ETC - Loop 2 start at 2018-03-19 14:23:39.275287
ETC - Loop 2 end
ETC - Loop 3 start at 2018-03-19 14:24:29.360197
ETC - Loop 3 end
ETC - Saving data start at 2018-03-19 14:29:57.322016
ETC - Saved data
QTUM - Loop 1 start at 2018-03-19 14:30:00.398498
QTUM - Loop 1 end
QTUM - Loop 2 start at 2018-03-19 14:30:18.829285
Can't search much long ago data - QTUM.
QTUM - Loo

Can't search long ago data - XMR.
XMR - Loop 1 end
XMR - Saving data start at 2018-03-19 16:05:29.106044
XMR - Saved data
ETC - Loop 1 start at 2018-03-19 16:05:32.168974
ETC - Loop 1 end
ETC - Saving data start at 2018-03-19 16:06:14.411742
ETC - Saved data
QTUM - Loop 1 start at 2018-03-19 16:06:17.499413
Can't search long ago data - QTUM.
QTUM - Loop 1 end
QTUM - Saving data start at 2018-03-19 16:06:20.619684
QTUM - Saved data
BTG - Loop 1 start at 2018-03-19 16:06:23.669556
Can't search long ago data - BTG.
BTG - Loop 1 end
BTG - Saving data start at 2018-03-19 16:06:28.802401
BTG - Saved data
ZEC - Loop 1 start at 2018-03-19 16:06:31.839357
Can't search long ago data - ZEC.
ZEC - Loop 1 end
ZEC - Saving data start at 2018-03-19 16:06:34.957774
ZEC - Saved data
BTC - Loop 1 start at 2018-03-19 16:06:38.016739
Can't search long ago data - BTC.
BTC - Loop 1 end
BTC - Loop 2 start at 2018-03-19 16:06:41.130688
Can't search much long ago data - BTC.
BTC - Loop 2 end
BTC - Loop 3 start

BCH - Loop 3 end
BCH - Saving data start at 2018-03-19 17:41:20.333220
BCH - Saved data
LTC - Loop 1 start at 2018-03-19 17:41:23.403706
Can't search long ago data - LTC.
LTC - Loop 1 end
LTC - Loop 2 start at 2018-03-19 17:41:26.575323
Can't search much long ago data - LTC.
LTC - Loop 2 end
LTC - Loop 3 start at 2018-03-19 17:41:29.699492
LTC - Loop 3 end
LTC - Saving data start at 2018-03-19 17:48:07.748347
LTC - Saved data
EOS - Loop 1 start at 2018-03-19 17:48:10.836261
Can't search long ago data - EOS.
EOS - Loop 1 end
EOS - Loop 2 start at 2018-03-19 17:48:17.074623
Can't search much long ago data - EOS.
EOS - Loop 2 end
EOS - Loop 3 start at 2018-03-19 17:48:20.164726
EOS - Loop 3 end
EOS - Saving data start at 2018-03-19 17:49:58.566642
EOS - Saved data
DASH - Loop 1 start at 2018-03-19 17:50:01.598818
Can't search long ago data - DASH.
DASH - Loop 1 end
DASH - Loop 2 start at 2018-03-19 17:50:04.689246
Can't search much long ago data - DASH.
DASH - Loop 2 end
DASH - Loop 3 sta

QTUM - Loop 3 end
QTUM - Saving data start at 2018-03-19 19:33:59.898815
QTUM - Saved data
BTG - Loop 1 start at 2018-03-19 19:34:02.949188
Can't search long ago data - BTG.
BTG - Loop 1 end
BTG - Loop 2 start at 2018-03-19 19:34:05.997288
Can't search much long ago data - BTG.
BTG - Loop 2 end
BTG - Loop 3 start at 2018-03-19 19:34:09.126825
BTG - Loop 3 end
BTG - Saving data start at 2018-03-19 19:36:14.632911
BTG - Saved data
ZEC - Loop 1 start at 2018-03-19 19:36:17.667527
Can't search long ago data - ZEC.
ZEC - Loop 1 end
ZEC - Loop 2 start at 2018-03-19 19:36:20.929290
Can't search much long ago data - ZEC.
ZEC - Loop 2 end
ZEC - Loop 3 start at 2018-03-19 19:36:24.453203
ZEC - Loop 3 end
ZEC - Saving data start at 2018-03-19 19:40:03.578838
ZEC - Saved data
BTC - Loop 1 start at 2018-03-19 19:40:06.629544
Can't search long ago data - BTC.
BTC - Loop 1 end
BTC - Saving data start at 2018-03-19 19:40:09.785980
BTC - Saved data
ETH - Loop 1 start at 2018-03-19 19:40:12.894449
Can't

Can't search long ago data - XMR.
XMR - Loop 1 end
XMR - Saving data start at 2018-03-19 20:47:20.345201
XMR - Saved data
ETC - Loop 1 start at 2018-03-19 20:47:23.415129
Can't search long ago data - ETC.
ETC - Loop 1 end
ETC - Saving data start at 2018-03-19 20:47:26.440430
ETC - Saved data
QTUM - Loop 1 start at 2018-03-19 20:47:29.548040
Can't search long ago data - QTUM.
QTUM - Loop 1 end
QTUM - Saving data start at 2018-03-19 20:47:32.574665
QTUM - Saved data
BTG - Loop 1 start at 2018-03-19 20:47:35.626825
Can't search long ago data - BTG.
BTG - Loop 1 end
BTG - Saving data start at 2018-03-19 20:47:38.661992
BTG - Saved data
ZEC - Loop 1 start at 2018-03-19 20:47:41.706291
Can't search long ago data - ZEC.
ZEC - Loop 1 end
ZEC - Saving data start at 2018-03-19 20:47:44.747463
ZEC - Saved data
BTC - Loop 1 start at 2018-03-19 20:47:47.812968
Can't search long ago data - BTC.
BTC - Loop 1 end
BTC - Loop 2 start at 2018-03-19 20:47:50.838466
Can't search much long ago data - BTC.
B

KeyboardInterrupt: 